In [1]:
using JuMP
using Clp
using Distributions
using Random
using CSV
using DataFrames

In [2]:
beta0 = 3.1470
beta1 = 0.0459
beta2 = 0.1841

0.1841

In [2]:
beta0 = 3.8532
beta1 = 0.0442
beta2 = 0.1812

0.1812

In [4]:
beta0 = 4.528051767181878
beta1 = 0.04125646 
beta2 = 0.17162406

0.17162406

### Linear Programming

In [6]:
# Build model
model = Model(Clp.Optimizer)

@variables(model, begin
# Stage 1 Variables
    0.7<=x1<=296.4
    0<=x2<=49.6
# Stage 2 Variables
    yA>=0
    yB>=0
end)

# constraint
@constraint(model, b, x1+x2 <= 200)
@constraint(model, c, x1-0.5*x2 >= 0)
@constraint(model, d, yA <= 8)
@constraint(model, e, 2*yB <= 24)
@constraint(model, f, 3*yA+2*yB <= 36)
@constraint(model, g, -beta1*x1-beta2*x2+yA+yB <= beta0)

@objective(model, Max, -0.1*x1-0.5*x2+3*yA+5*yB)

# Optimization
optimize!(model)

# Assert problem is solved
# @assert(termination_status(model) == OPTIMAL)

@show value.(x1)
@show value.(x2)
@show objective_value(model)

value.(x1) = 199.99999999999997
value.(x2) = 0.0
objective_value(model) = 42.33803130154563


42.33803130154563

Coin0506I Presolve 4 (-2) rows, 4 (0) columns and 10 (-2) elements
Clp0006I 0  Obj -0.07 Dual inf 1.567538 (2)
Clp0006I 3  Obj 42.338031
Clp0000I Optimal - objective value 42.338031
Coin0511I After Postsolve, objective 42.338031, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 42.3380313 - 3 iterations time 0.002, Presolve 0.00


### Stochastic Decomposition

In [ ]:
ENV["CPLEX_STUDIO_BINARIES"] = "C:/Cplex/cplex/bin/x64_win64"
import Pkg
Pkg.add("CPLEX")
Pkg.build("CPLEX")

In [3]:
using CPLEX
using TwoSD

In [4]:
e_train_data = CSV.read("e_train.csv", DataFrame)
e_train = e_train_data[:,2]

100-element Vector{Float64}:
  1.442155908879112
 -2.023543306261276
 -3.0869473257074542
  0.801840808517543
 -0.5259086789331775
 -5.349559902483894
 -0.022840695968245228
  0.9325506498261902
  0.872010514252465
 -2.1873194051951277
  1.3541531692602735
 -0.010787994265751166
 -1.5010481438369645
  ⋮
  1.008961157088093
 -0.2828545064779284
  0.9926275716909423
  2.565406426682827
  0.10247595682539412
  0.8277552192856028
  0.8505729580154977
  0.4467636713809924
 -1.1521520476829092
  0.00802943999837602
  1.180706457833665
  0.175610141324114

In [6]:
model = direct_model(CPLEX.Optimizer())

@variables(model, begin
# Stage 1 Variables
    x1>=0.7
    x2>=0
# Stage 2 Variables
    yA>=0
    yB>=0
end)

# constraint
@constraint(model, b, x1+x2-200 <= 0)
@constraint(model, c, -x1+0.5*x2 <= 0)
@constraint(model, i1, x1<=296.4)
@constraint(model, i2, x2<=49.6)

@constraint(model, d, yA-8 <= 0)
@constraint(model, e, 2*yB-24 <= 0)
@constraint(model, f, 3*yA+2*yB-36 <= 0)
@constraint(model, g, -beta1*x1-beta2*x2+yA+yB <= 0) #beta0

@objective(model, Min, 0.1*x1+0.5*x2-3*yA-5*yB)

split_position = Position(d, yA)
r = Random.MersenneTwister(1234)
function mystoc()
    e = rand(r, e_train)
    binding = [Position(g, "RHS") => e+beta0]
    return OneRealization(binding)
end

user_mean = [beta0]

solve_sd(model, split_position, user_mean, mystoc)

: 

: 

In [ ]:
rand(r, e_train)